In [ ]:
import psycopg
import numpy as np
from sentence_transformers import SentenceTransformer

# ===== CONFIGURATION =====
conversation_file_path = "023_00000018.txt"

db_params = {
    'dbname': 'rag_chatbot',
    'user': 'postgres',
    'password': '11649303', 
    'host': 'localhost',
    'port': '5432'
}

print("Chargement du modèle d'embedding local")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(" Modèle chargé !")

def create_conversation_list(file_path: str) -> list[str]:
    """Lit le fichier texte et retourne une liste de phrases"""
    with open(file_path, "r", encoding="latin-1") as file:
        text_list = [ligne.strip().removeprefix("     ") for ligne in file.readlines() 
                     if ligne.strip() and not ligne.startswith("<")]
    return text_list

def calculate_embeddings(corpus: str) -> list[float]:
    """Calcule l'embedding d'un texte avec le modèle local"""
    embedding = model.encode(corpus)
    return embedding.tolist()

def cosine_similarity(vec1, vec2):
    """Calcule la similarité entre deux vecteurs"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def save_embedding(corpus: str, embedding: list[float], cursor) -> None:
    """Sauvegarde un texte et son embedding dans la base"""
    cursor.execute("INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)", 
                   (corpus, embedding))

def similar_corpus(input_corpus: str, top_k: int = 5) -> list[tuple[int, str, float]]:
    """Trouve les textes les plus similaires à input_corpus"""
    input_embedding = calculate_embeddings(input_corpus)
    
    conn_str = f"dbname={db_params['dbname']} user={db_params['user']} password={db_params['password']} host={db_params['host']} port={db_params['port']}"
    
    with psycopg.connect(conn_str) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT id, corpus, embedding FROM embeddings")
            results = cur.fetchall()
            
            similarities = []
            for id, corpus, embedding in results:
                similarity = cosine_similarity(input_embedding, embedding)
                similarities.append((id, corpus, similarity))
            
            similarities.sort(key=lambda x: x[2], reverse=True)
            return similarities[:top_k]

if __name__ == "__main__":
    conn_str = f"dbname={db_params['dbname']} user={db_params['user']} password={db_params['password']} host={db_params['host']} port={db_params['port']}"
    
    try:
        print(" Démarrage du programme")
        
        with psycopg.connect(conn_str) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                print(" Création de la table")
                cur.execute("DROP TABLE IF EXISTS embeddings")
                cur.execute("""
                    CREATE TABLE embeddings (
                        id SERIAL PRIMARY KEY,
                        corpus TEXT,
                        embedding FLOAT[]
                    )
                """)
                print("Table créée ")
                
                print(f"Lecture du fichier {conversation_file_path}")
                corpus_list = create_conversation_list(conversation_file_path)
                print(f" {len(corpus_list)} textes trouvés !")
                
                print(" Calcul des embeddings localement (rapide)")
                for i, corpus in enumerate(corpus_list, 1):
                    print(f"   [{i}/{len(corpus_list)}] Traitement")
                    embedding = calculate_embeddings(corpus)
                    save_embedding(corpus, embedding, cur)
                
                conn.commit()
                print("Tous les embeddings sont sauvegardés !\n")
        
    except FileNotFoundError:
        print(f" Erreur : Le fichier '{conversation_file_path}' n'existe pas !")
    except psycopg.OperationalError as e:
        print(f" Erreur de connexion à PostgreSQL :")
        print(f"   Vérifiez que PostgreSQL est démarré")
        print(f"   Vérifiez votre mot de passe")
        print(f"   Détails : {e}")
    except Exception as e:
        print(f" Erreur : {e}")

c:\Users\hp\OneDrive\Bureau\Chatbot-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📥 Chargement du modèle d'embedding local...
✅ Modèle chargé !
🚀 Démarrage du programme...
📊 Création de la table...
✅ Table créée !
📖 Lecture du fichier 023_00000018.txt...
✅ 14 textes trouvés !
⏳ Calcul des embeddings localement (rapide)...
   [1/14] Traitement...
   [2/14] Traitement...
   [3/14] Traitement...
   [4/14] Traitement...
   [5/14] Traitement...
   [6/14] Traitement...
   [7/14] Traitement...
   [8/14] Traitement...
   [9/14] Traitement...
   [10/14] Traitement...
   [11/14] Traitement...
   [12/14] Traitement...
   [13/14] Traitement...
   [14/14] Traitement...
✅ Tous les embeddings sont sauvegardés !



In [ ]:
from transformers import pipeline

# Charger un modèle de génération de texte
print(" Chargement du modèle de génération ")
generator = pipeline('text2text-generation', model='google/flan-t5-base')
print(" Modèle de génération chargé !")

def chatbot_simple(question: str, top_k: int = 3):
    """Chatbot simple qui affiche les résultats pertinents"""
    
    print(f"\nRecherche pour : '{question}'\n")
    results = similar_corpus(question, top_k=top_k)
    
    print(" Voici les extraits les plus pertinents :\n")
    print("=" * 70)
    
    for i, (id, corpus, score) in enumerate(results, 1):
        print(f"\n{i}. Pertinence: {score:.1%}")
        print(f"   {corpus}")
    
    print("\n" + "=" * 70)

# TEST
print("CHATBOT RAG\n")

while True:
    question = input(" Votre question : ")
    
    if question.lower() in ['quit', 'exit', 'q']:
        print(" Au revoir !")
        break
    
    if question.strip():
        chatbot_simple(question, top_k=3)

📥 Chargement du modèle de génération (cela peut prendre du temps la première fois)...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\hp\OneDrive\Bureau\Chatbot-RAG\venv\Lib\site-packages\huggingface_hub\file_download.py:798: UserWarning: Not enough free disk space to download the file. The expected file size is: 0.79 MB. The target location C:\Users\hp\.cache\huggingface\hub\models--google--flan-t5-base\blobs only has 0.58 MB free disk space.
  warnings.warn(
c:\Users\hp\OneDrive\Bureau\Chatbot-RAG\venv\Lib\site-packages\huggingface_hub\file_download.py:798: UserWarning: Not enough free disk space to download the file. The expected file size is: 0.79 MB. The target location C:\Users\hp\.cache\huggingface\hub\models--google--flan-t5-base\blobs only has 0.50 MB free disk space.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling

✅ Modèle de génération chargé !
🎯 CHATBOT RAG


🔍 Recherche pour : 'qu elle est le numero de telephone'

🤖 Voici les extraits les plus pertinents :


1. Pertinence: 80.1%
   c: ah ben écoutez j'ai j'ai e je sais pas j'ai simplement le numéro de téléphone e sur lequel je vous ai appelé

2. Pertinence: 47.3%
   c: je e je suis désolée j'ai simplement le 0 2 97 0 1 27 25

3. Pertinence: 46.3%
   h: vous pouvez me dire le bureau ou alors e


🔍 Recherche pour : 'numéro de téléphone'

🤖 Voici les extraits les plus pertinents :


1. Pertinence: 75.3%
   c: ah ben écoutez j'ai j'ai e je sais pas j'ai simplement le numéro de téléphone e sur lequel je vous ai appelé

2. Pertinence: 41.9%
   c: je e je suis désolée j'ai simplement le 0 2 97 0 1 27 25

3. Pertinence: 38.6%
   c: oui bonjour excusez moi de vous déranger est ce que c'est possible d'avoir Prénom Nom s'il vous plaît

👋 Au revoir !
